<a href="https://colab.research.google.com/github/Jojo666/PKNP-Data/blob/master/SVM_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SVM classification: https://github.com/giswqs/earthengine-py-notebooks/blob/master/MachineLearning/svm_classifier.ipynb

In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=rdL3acQqOSl5axrsPVQeGdr7iFsef3H526_fSzmf_Zk&tc=T7S4iHvavBzM8fARLDU07qjqRubMQt7HVE9KXRKrkLs&cc=SurTQKiUEVIfPYpJj_O0LFXzkiPm2YlVM1cEfqI8h-o

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh7cNOm6Le_2dR0CGFNfkqum7KMEwgcwLXkCaGXUGkCgz3kkieKmY8

Successfully saved authorization token.


In [3]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [19]:
#Map = geemap.Map(center=[40,-100], zoom=4)
#Map

In [30]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))

In [31]:
# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

In [32]:
image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
}).clip(aoi)

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [33]:
# Manually created polygons.
forest1 = ee.Geometry.Rectangle(-63.0187, -9.3958, -62.9793, -9.3443)
forest2 = ee.Geometry.Rectangle(-62.8145, -9.206, -62.7688, -9.1735)
nonForest1 = ee.Geometry.Rectangle(-62.8161, -9.5001, -62.7921, -9.4486)
nonForest2 = ee.Geometry.Rectangle(-62.6788, -9.044, -62.6459, -8.9986)

In [34]:
# Make a FeatureCollection from the hand-made geometries.
polygons = ee.FeatureCollection([
  ee.Feature(nonForest1, {'class': 0}),
  ee.Feature(nonForest2, {'class': 0}),
  ee.Feature(forest1, {'class': 1}),
  ee.Feature(forest2, {'class': 1}),
])

In [35]:
# Get the values for all pixels in each polygon in the training.
training = image.sampleRegions(**{
  # Get the sample from the polygons FeatureCollection.
  'collection': polygons,
  # Keep this list of properties from the polygons.
  'properties': ['class'],
  # Set the scale to get Landsat pixels in the polygons.
  'scale': 30
})


In [36]:
# Create an SVM classifier with custom parameters.
classifier = ee.Classifier.libsvm(**{
  'kernelType': 'RBF',
  'gamma': 0.5,
  'cost': 10
})

In [37]:
# Train the classifier.
trained = classifier.train(training, 'class', bands)


In [38]:
# Classify the image.
classified = image.classify(trained)

In [39]:
# Display the classification result and the input image.
Map.setCenter(-62.836, -9.2399, 9)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.5, 'gamma': 2})
Map.addLayer(polygons, {}, 'training polygons')
Map.addLayer(classified,
             {'min': 0, 'max': 1, 'palette': ['red', 'green']},
             'deforestation')

In [40]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(bottom=69215.0, center=[-9.2399, -62.836], controls=(WidgetControl(options=['position', 'transparent_bg'],…